## 使用套件

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from bs4 import BeautifulSoup
from urllib.parse import quote
import re
import requests
import sys
import os
import time
import json
import html2text
import pandas as pd
import undetected_chromedriver as uc

#### URL列表

In [ ]:
# url列表
url_list = ['https://www.google.com.tw/maps/place/%E5%87%AANagi+%E5%BF%A0%E5%AD%9D%E5%BA%97/@25.0423245,121.5443867,17z/data=!3m2!4b1!5s0x3442abda94a4106d:0x7ec6f5a0b4d6d0d!4m5!3m4!1s0x3442abda93fa18e9:0x3862944b40c0c5b8!8m2!3d25.0423197!4d121.5465754?hl=zh-TW']

## 店家詳細資訊

In [33]:
def initialize_chrome(_from="facebook",retry=0):
    global driver 
    try:
        print("Initializing chromedriver.")
        options = Options()
#         options.add_argument(f'--user-data-dir={google_session_profile}')
        options.add_argument("--start-maximized")
        options.add_argument("--incognito")
        options.add_argument("--disable-gpu")
        options.add_argument('--log-level=3')
        #options.add_argument('--blink-settings=imagesEnabled=false')
        driver = uc.Chrome(options=options)
        time.sleep(2)
        return True
    except Exception as e:
        print(e)
        pass

    
    
initialize_chrome()
driver.get(url_list[0])
response = driver.page_source
soup = BeautifulSoup(response, 'html.parser')


rating_and_review_div = soup.find('div',{'class':'k7jAl lJ3Kh miFGmb'})
# print(rating_and_review_div)


# 店名
alias = rating_and_review_div.find(class_="fontHeadlineLarge").text.strip()
print('店名: ' + alias)

# 評論數
total_reviews = rating_and_review_div.find('span',{'aria-label':re.compile("^\d.+ 則評論")})
if total_reviews:
    total_reviews = total_reviews.get('aria-label','').strip() 
    total_reviews = total_reviews.removesuffix('則評論').strip()
if not total_reviews:
    total_reviews = ""
print('評論數: ' + total_reviews)

# 星級
total_rating = rating_and_review_div.find('span',{'aria-label':re.compile("^\s+?\d+\.\d+ 星級")})
if total_rating:
    total_rating = total_rating.get('aria-label','').strip() 
    total_rating = total_rating.removesuffix('星級').strip()
if not total_rating:
    total_rating = ""
print('星級: ' + total_rating)

# 標籤
category = rating_and_review_div.find('button',{'jsaction':re.compile("pane.rating.category")})
if category:
    category = category.getText()
if not category:
    category = ""
print('標籤: ' + category)

# 價位
price = rating_and_review_div.find('span',{'aria-label':re.compile("^價格")})
if price:
    price = price.getText()
if not price:
    price = ""
print('價位: ' + price)

# 地址
address = rating_and_review_div.find('button',{'aria-label':re.compile("^地址\:")})
if address:
    address = address.get('aria-label','').strip() 
    address = address.removeprefix('地址:')      
if not address:
    address = ""
print('地址: ' + address)

# 電話
phone = rating_and_review_div.find('button',{'aria-label':re.compile("^電話號碼\:")})
if phone:
    phone = phone.get('aria-label','').strip()    
    phone = phone.removeprefix('電話號碼:')
if not phone:
    phone = ""
print('電話: ' + phone)

# 營業時間
opening_hours = rating_and_review_div.find('div',{'aria-label':re.compile("(星期.、(24 小時營業)?(休息)?(\d+:\d+ 到 \d+:\d+(、\d+:\d+ 到 \d+:\d+)?)?;? ?){7}")})
if opening_hours:
    opening_hours = opening_hours.get('aria-label','')   
if not opening_hours:
    opening_hours = ""
print('營業時間: ' + opening_hours)

# 網站
website = rating_and_review_div.find('a',{'aria-label':re.compile("^網站\:")})
if website:
    website = website.get('href')   
if not website:
    website = ""
print('網站: ' + website)

Initializing chromedriver.
店名: 凪Nagi 忠孝店
評論數: 3,543
星級: 4.3
標籤: 拉麵店
價位: $$
地址:  106台北市大安區大安路一段75巷5號1
電話:  02 2778 1899
營業時間: 星期五、11:30 到 20:00; 星期六、11:30 到 20:00; 星期日、11:30 到 20:00; 星期一、11:30 到 20:00; 星期二、11:30 到 20:00; 星期三、11:30 到 20:00; 星期四、11:30 到 20:00. 隱藏本週營業時間
網站: https://www.facebook.com/NagiTaiwan/


## 評論爬取

In [10]:
def parse_description(description_tag):
    description_text = h.handle(str(description_tag)) 
    return description_text
h = html2text.HTML2Text()
h.ignore_links = True

# 下一頁review內容的token(寫進url)
next_page_token = ""

# 設定biz_id(寫進url)
biz_id = re.search(r"1s(0.*?\:.*?)[^a-zA-Z\d\s:]",url_list[0]) # 寫成迴圈的時候要改一下
if not biz_id:
    print("Not a valid url.")
biz_id = biz_id.groups()[0]

reviewData = []

while True:
    
    print('已下載', len(reviewData))
    url = f'https://www.google.com/async/reviewSort?yv=3&async=feature_id:{biz_id},review_source:All%20reviews,sort_by:newestFirst,is_owner:false,filter_text:,associated_topic:,next_page_token:{next_page_token},_pms:s,_fmt:json'
    response = requests.get(url)

    response = response.text.removeprefix(")]}'")
    json_data = json.loads(response)["localReviewsProto"]
    review_data = json_data["other_user_review"]

    for result in review_data:
        rating = result['star_rating']['value']
        profile_pic = result['profile_photo_url']
        profile_pic = re.sub('=s(\d+)-','=s180-',profile_pic)
        reviewer_name = result['author_real_name']
        review_date = result['publish_date']['localized_date']      
        if result.get('review_text'):
            review_text = parse_description(result['review_text']['full_html'])
        else:
            review_text = ""

        info_dict = {}        
        info_dict['author'] = reviewer_name
        info_dict['rating'] =  rating             
        info_dict["date"] = review_date
        info_dict["text"] = review_text

        reviewData.append(info_dict)

    next_page_token = json_data.get('next_page_token','').strip()
    if not next_page_token:
        break
        
reviews_info_header = list(reviewData[0].keys())
reviews_info_df = pd.DataFrame.from_records(reviewData,columns=reviews_info_header)
reviews_info_df.to_csv('./test.csv')

已下載 0
已下載 10
已下載 20
已下載 30
已下載 40
已下載 50
已下載 60
已下載 70
已下載 80
已下載 90
已下載 100
已下載 110
已下載 120
已下載 130
已下載 140
已下載 150
已下載 160
已下載 170
已下載 180
已下載 190
已下載 200
已下載 210
已下載 220
已下載 230
已下載 240
已下載 250
已下載 260
已下載 270
已下載 280
已下載 290
已下載 300
已下載 310
已下載 320
已下載 330
已下載 340
已下載 350
已下載 360
已下載 370
已下載 380
已下載 390
已下載 400
已下載 410
已下載 420
已下載 430
已下載 440
已下載 450
已下載 460
已下載 470
已下載 480
已下載 490
已下載 500
已下載 510
已下載 520
已下載 530
已下載 540
已下載 550
已下載 560
已下載 570
已下載 580
已下載 590
已下載 600
已下載 610
已下載 620
已下載 630
已下載 640
已下載 650
已下載 660
已下載 670
已下載 680
已下載 690
已下載 700
已下載 710
已下載 720
已下載 730
已下載 740
已下載 750
已下載 760
已下載 770
已下載 780
已下載 790
已下載 800
已下載 810
已下載 820
已下載 830
已下載 840
已下載 850
已下載 860
已下載 870
已下載 880
已下載 890
已下載 900
已下載 910
已下載 920
已下載 930
已下載 940
已下載 950
已下載 960
已下載 970
已下載 980
已下載 990
已下載 1000
已下載 1010
已下載 1020
已下載 1030
已下載 1040
已下載 1050
已下載 1060
已下載 1070
已下載 1080
已下載 1090
已下載 1100
已下載 1110
已下載 1120
已下載 1130
已下載 1140
已下載 1150
已下載 1160
已下載 1170
已下載 1180
已下載 1190
已下載 1200
已下載 1210
已下載 1220
已下載 